In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import re

In [2]:
def processData():
    df = pd.read_csv("data/person.csv")
    df = df[df["emptype"] != "Not in Work Force"]
    medianImputer = SimpleImputer(strategy="median")

    df = createWFHBin(df)
    df["employment_type"] = df["mainact"].apply(processEmployment)
    df["carlicence"] = df["carlicence"].apply(processlicence)

    df[["persinc", "persinc_group_median"]] = df["persinc"].apply(lambda x: pd.Series(processIncome(x)))
    df[["persinc_group_median"]] = medianImputer.fit_transform(df[["persinc_group_median"]])

    df["agegroup"] = df["agegroup"].apply(processAge)
    df["industry"] = df["anzsic1"].apply(processIndustry)
    df["occupation"] = df["anzsco1"].apply(processOccupation)

    df = df.dropna(subset=["agegroup", "sex", "employment_type", "carlicence", "industry", "occupation"])
    df = createIncomeBin(df)
    df = oneHotEncode(df)
    df = labelEncode(df)

    df.to_csv("data/cleaned_person.csv", index=False)
    print(df)

In [3]:
def createWFHBin(df):
    days = ["wfhmon", "wfhtue", "wfhwed", "wfhthu", "wfhfri", "wfhsat", "wfhsun"]

    df["wfh_days"] = (df[days] == "Yes").sum(axis=1)
    df["wfh_freq"] = df["wfh_days"].apply(lambda x: "Never" if x == 0 else "Occasional" if x < 3 else "Frequent")

    return df

In [4]:
def processIndustry(industry):
    info_based = [
        "Financial and Insurance Services",
        "Professional, Scientific and Technical Services",
        "Public Administration and Safety",
        "Information Media and Telecommunications",
        "Administrative and Support Services",
    ]

    physical = [
        "Agriculture, Forestry and Fishing",
        "Mining",
        "Manufacturing",
        "Electricity, Gas, Water and Waste Services",
        "Construction",
        "Transport, Postal and Warehousing",
        "Wholesale Trade",
    ]

    public_facing = [
        "Retail Trade",
        "Accommodation and Food Services",
        "Rental, Hiring and Real Estate Services",
        "Health Care and Social Assistance",
        "Arts and Recreation Services",
        "Education and Training",
    ]

    if industry in info_based:
        return "Information-Based"
    elif industry in physical:
        return "Physical"
    elif industry in public_facing:
        return "Public-Facing"
    else:
        return "Other"


In [5]:
def processOccupation(occupation):
    office = [
        "Professionals",
        "Clerical and Administrative Workers",
        "Managers",
    ]

    on_site = [
        "Technicians and Trades Workers",
        "Machinery Operators and Drivers",
        "Labourers",
    ]

    service = [
        "Sales Workers",
        "Community and Personal Service Workers",
    ]

    if occupation in office:
        return "Office"
    elif occupation in on_site:
        return "On-Site/Manual"
    elif occupation in service:
        return "Service"
    else:
        return "Other"

In [6]:
def createIncomeBin(df):
    df["persinc_bin"] = pd.qcut(
        df["persinc_group_median"],
        q=3,
        labels=["Low", "Medium", "High"],
        duplicates="drop"
    )

    df["persinc_bin_le"] = LabelEncoder().fit_transform(df["persinc_bin"])

    return df

In [7]:
def processEmployment(type):
    if "Full-time Work" in type:
        return "Full-time"
    elif "Part-time Work" in type:
        return "Part-time"
    elif "Casual Work" in type:
        return "Casual"
    else:
        return "Other"

In [8]:
def processlicence(type):
    if type in ["Full Licence", "Red Probationary Licence", "Green Probationary Licence", "Learners Permit"]:
        return "Has licence"
    else:
        return "No licence"

In [9]:
def processIncome(value):
    if value == "Nil income":
        return "0-0", 0
    if value == "Negative income":
        return "negative", -1

    match = re.search(r"\((\$?[\d,]+)-(\$?[\d,]+)\)", value)
    if match:
        low = int(match.group(1).replace("$", "").replace(",", ""))
        high = int(match.group(2).replace("$", "").replace(",", ""))
        yearly_str = f"{low}-{high}"
        midpoint = (low + high) / 2
        return yearly_str, midpoint

    match = re.search(r"\((\$?[\d,]+) or more\)", str(value))
    if match:
        low = int(match.group(1).replace("$", "").replace(",", ""))
        yearly_str = f"{low}-inf"
        return yearly_str, low

    return value, None

In [10]:
def processAge(age):
    match = re.search(r"(\d+)->(\d+)", str(age))
    if match:
        lower = int(match.group(1))

        if lower < 20:
            return "<20"
        elif 20 <= lower < 30:
            return "20->29"
        elif 30 <= lower < 40:
            return "30->39"
        elif 40 <= lower < 50:
            return "40->49"
        else:
            return "50+"
    return None

In [11]:
def labelEncode(df):
    features = ["wfh_freq", "occupation", "industry", "agegroup", "sex", "carlicence", "employment_type"]

    for feature in features:
        labelEncoder = LabelEncoder()
        df[f"{feature}_le"] = labelEncoder.fit_transform(df[feature])

    return df

In [12]:
def oneHotEncode(df):
    features = ["occupation", "industry", "agegroup", "sex", "carlicence", "employment_type"]

    for feature in features:
        encoded = pd.get_dummies(df[feature], prefix=feature, drop_first=False, dtype=int)
        df = pd.concat([df, encoded], axis=1)

    return df

In [13]:
processData()

              persid         hhid  persno    travdow agegroup     sex  \
1     Y24H5740102P02  Y24H5740102       2    Tuesday   20->29    Male   
3     Y24H5740104P01  Y24H5740104       1   Thursday      50+  Female   
4     Y24H5740109P01  Y24H5740109       1    Tuesday   40->49    Male   
5     Y24H5740109P02  Y24H5740109       2    Tuesday   40->49  Female   
11    Y24H5740110P03  Y24H5740110       3  Wednesday   20->29  Female   
...              ...          ...     ...        ...      ...     ...   
8164  Y24H6120115P01  Y24H6120115       1     Monday   20->29    Male   
8165  Y24H6120115P02  Y24H6120115       2     Monday   20->29  Female   
8168  Y24H6180402P02  Y24H6180402       2    Tuesday   20->29  Female   
8169  Y24H6210434P01  Y24H6210434       1   Saturday   40->49  Female   
8170  Y24H6210434P02  Y24H6210434       2   Saturday   40->49    Male   

     relationship   carlicence mbikelicence otherlicence  ...  \
1          Spouse  Has licence           No           No  